# Data Preprocessing

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')

train

,Artist,Song,Genre,Language,Lyrics
0,12 stones,world so cold,Rock,en,"It starts with pain, followed by hate\nFueled ..."
1,12 stones,broken,Rock,en,Freedom!\nAlone again again alone\nPatiently w...
2,12 stones,3 leaf loser,Rock,en,"Biting the hand that feeds you, lying to the v..."
3,12 stones,anthem for the underdog,Rock,en,You say you know just who I am\nBut you can't ...
4,12 stones,adrenaline,Rock,en,My heart is beating faster can't control these...
...,...,...,...,...,...
290178,bobby womack,i wish he didn t trust me so much,R&B,en,I'm the best friend he's got I'd give him the ...
290179,bad boys blue,i totally miss you,Pop,en,"Bad Boys Blue ""I Totally Miss You"" I did you w..."
290180,celine dion,sorry for love,Pop,en,Forgive me for the things That I never said to...
290181,dan bern,cure for aids,Indie,en,The day they found a cure for AIDS The day the...


In [ ]:
train['Genre'].value_counts()

Rock          121404
Pop           108714
Metal          20291
Jazz           13545
Folk            8644
Indie           8449
R&B             2793
Hip-Hop         2240
Electronic      2213
Country         1890
Name: Genre, dtype: int64

In [ ]:
en_data = train[train['Language']=='en']
rock = en_data[en_data['Genre']=='Rock']

rock.shape

(107145, 5)

In [ ]:
print(rock.isnull().sum())
rock = rock[rock['Lyrics'].notna()]


Artist      0
Song        1
Genre       0
Language    0
Lyrics      0
dtype: int64


In [ ]:
rock_sub = rock.sample(n=500)



rock_sub.shape

(500, 5)

In [ ]:
rock_lyrics = rock_sub['Lyrics'].astype('str')

rock_lyrics

248362    Well mama go a tippy tippy toein' through the ...
26481     Take my photo off the wall\nIf it just won't s...
196443    I wanna dance by water 'neath the Mexican sky\...
61472     You want to cuss and fight\nIn the basement\nT...
214203    I found him by the railroad track this morning...
                                ...                        
245665    There are people who don't know me\nWho must t...
64984     I look up\nThere are dark flames comin' down\n...
194296    There'll be no strings to bind your hands\nNot...
94212     Though I know it's time to take you home\nBaby...
62414     Life's a bitch, then you die\nBlack Hell\nHell...
Name: Lyrics, Length: 500, dtype: object

In [ ]:
rock_lyrics[:2]

248362    Well mama go a tippy tippy toein' through the ...
26481     Take my photo off the wall\nIf it just won't s...
Name: Lyrics, dtype: object

In [ ]:
rocklist = list(rock_lyrics)

rocklist[:2]

["Well mama go a tippy tippy toein' through the house\nGotta see what's the matter with the baby\nIt's been a bawlin' and a squallin' and a kickin' on the wall\nWell I guess it's dropped its little bottle maybe\nOh daddy come a runnin' with the water and a rag\nGonna need another little diaper maybe\nNo need a hesitatin' or a wonderin' and a waitin' I know what's the matter with baby\nOh mama rock a baby daddy rock a baby too\nIt's a yawnin' and a gettin' sleepy maybe\nSo mama keep a rockin' daddy rock a little too and at last it's a sleepin' little baby\n[ steel ]\nWell mama tippy toein' daddy tippy toein' too gonna try to do a little sleepin' maybe\nWhether it'll be a minute or an hour or two is depending entirely on the baby\nWell I hear the clock a ringin' just a dongin' and a dingin'\nDaddy go and make a little money maybe\nMama sleep a little longer but remember while he's gone\nWon't you please take care of the baby\nOh mama rock a baby...\nAnd at last it's a sleepin' little bab

In [ ]:
import string

def clean_text(txt):
    txt = "".join(i for i in txt if i not in string.punctuation).lower()
    txt = txt.replace("\n"," ")
    return txt 

corpus = [clean_text(x) for x in rocklist]
corpus[0]

'well mama go a tippy tippy toein through the house gotta see whats the matter with the baby its been a bawlin and a squallin and a kickin on the wall well i guess its dropped its little bottle maybe oh daddy come a runnin with the water and a rag gonna need another little diaper maybe no need a hesitatin or a wonderin and a waitin i know whats the matter with baby oh mama rock a baby daddy rock a baby too its a yawnin and a gettin sleepy maybe so mama keep a rockin daddy rock a little too and at last its a sleepin little baby  steel  well mama tippy toein daddy tippy toein too gonna try to do a little sleepin maybe whether itll be a minute or an hour or two is depending entirely on the baby well i hear the clock a ringin just a dongin and a dingin daddy go and make a little money maybe mama sleep a little longer but remember while hes gone wont you please take care of the baby oh mama rock a baby and at last its a sleepin little baby after all its a sleepin little baby after all its a

In [ ]:
len(corpus[0])

1056

In [ ]:
len(corpus)

500

In [ ]:
corpus

### Input Sequences

Input sequence is the numerical representation of how our words are arranged. 

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

In [ ]:
inp_sequences, total_words = get_sequence_of_tokens(corpus)


In [ ]:
inp_sequences[:10]

[[61, 630],
 [61, 630, 40],
 [61, 630, 40, 6],
 [61, 630, 40, 6, 1726],
 [61, 630, 40, 6, 1726, 1726],
 [61, 630, 40, 6, 1726, 1726, 2061],
 [61, 630, 40, 6, 1726, 1726, 2061, 114],
 [61, 630, 40, 6, 1726, 1726, 2061, 114, 1],
 [61, 630, 40, 6, 1726, 1726, 2061, 114, 1, 413],
 [61, 630, 40, 6, 1726, 1726, 2061, 114, 1, 413, 160]]

In [ ]:
total_words

6378

In [ ]:
tokenizer.word_index['everyday']

2378

### Pad Sequences

In [ ]:
from keras.preprocessing.sequence import pad_sequences

max_sequence_len = max([len(x) for x in inp_sequences])
inp_sequences = np.array(pad_sequences(inp_sequences,
                       maxlen = max_sequence_len, padding='pre'))

In [ ]:
max_sequence_len

824

In [ ]:
print(inp_sequences.shape)
inp_sequences

(101732, 824)


array([[   0,    0,    0, ...,    0,   61,  630],
       [   0,    0,    0, ...,   61,  630,   40],
       [   0,    0,    0, ...,  630,   40,    6],
       ...,
       [   0,    0,    0, ..., 1614, 2564,  884],
       [   0,    0,    0, ..., 2564,  884, 1492],
       [   0,    0,    0, ...,  884, 1492, 3526]], dtype=int32)

### Predictions and Labels

we will use our input sequence and use the last word of all sequences as labels for all previous words

In [ ]:
predictors, label = inp_sequences[:,:-1],inp_sequences[:,-1]

In [ ]:
import keras.utils as ku

label = ku.to_categorical(label, num_classes=total_words)

In [ ]:
label.shape

(101732, 6378)

## Build the Model

In [ ]:
from keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model01 = Sequential()
model01.add(Embedding(total_words, 50, input_length=max_sequence_len-1))
# Add a LSTM Layer
model01.add(LSTM(100)) 
model01.add(Dense(total_words/2, activation='relu'))  
# In the last layer, the shape should be equal to the total number of words present in our corpus
model01.add(Dense(total_words, activation='softmax'))
model01.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')  

model01.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 823, 50)           318900    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dense (Dense)                (None, 3189)              322089    
_________________________________________________________________
dense_1 (Dense)              (None, 6378)              20345820  
Total params: 21,047,209
Trainable params: 21,047,209
Non-trainable params: 0
_________________________________________________________________


In [ ]:
his01 = model01.fit(predictors, label, epochs= 10, verbose=1)

Epoch 1/10
3180/3180 [==============================] - 166s 42ms/step - loss: 6.3721 - accuracy: 0.0475
Epoch 2/10
3180/3180 [==============================] - 132s 41ms/step - loss: 5.3229 - accuracy: 0.1172
Epoch 3/10
3180/3180 [==============================] - 131s 41ms/step - loss: 4.5892 - accuracy: 0.1802
Epoch 4/10
3180/3180 [==============================] - 131s 41ms/step - loss: 3.9433 - accuracy: 0.2471
Epoch 5/10
3180/3180 [==============================] - 133s 42ms/step - loss: 3.3562 - accuracy: 0.3174
Epoch 6/10
3180/3180 [==============================] - 133s 42ms/step - loss: 2.8106 - accuracy: 0.3925
Epoch 7/10
3180/3180 [==============================] - 132s 42ms/step - loss: 2.3019 - accuracy: 0.4742
Epoch 8/10
3180/3180 [==============================] - 131s 41ms/step - loss: 1.8296 - accuracy: 0.5651
Epoch 9/10
3180/3180 [==============================] - 131s 41ms/step - loss: 1.4552 - accuracy: 0.6433
Epoch 10/10
3180/3180 [==============================] 

Save the model

In [ ]:
!pip install -q pyyaml h5py

model01.save('rock_enonly.h5')

### Lyrics Generation

In [ ]:
def make_lyrics(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],
                     maxlen=max_sequence_len-1,padding='pre')
        
        predicted = model01.predict_classes(token_list, verbose=0)

        output_word = ""
        
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text)

In [ ]:
make_lyrics("look at", 100)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


look at mei must be absolutely crazy how did 1 ever ever stop the news of time ill stand the sun was the night cause when the night that life was that lonesome comes through the anger and the laughter she knows that she matters and somebody out there cares what does she gain by their soul she found the boy she likes she wanted to get married settle down for life she didnt know love was so grand she never took love second hand but she called the soul of heaven run off the earth in their squeaky under mornings sleepy


In [ ]:
make_lyrics("look at", 200)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


look at mei must be absolutely crazy how did 1 ever ever stop the news of time ill stand the sun was the night cause when the night that life was that lonesome comes through the anger and the laughter she knows that she matters and somebody out there cares what does she gain by their soul she found the boy she likes she wanted to get married settle down for life she didnt know love was so grand she never took love second hand but she called the soul of heaven run off the earth in their squeaky under mornings sleepy were so glad were so far under far and early way haul away well haul away joe haul away joe boys meeting on up with the old st croix keep a rowing me boys its a raftmans life for me toy cars and candy bars are just a waste of money way haul away well haul away joe haul away joe boys meeting on up with the old st croix keep a rowing me boys its a raftmans life for me toy cars and candy bars are just a waste of money way haul away well haul away joe haul


In [ ]:
make_lyrics("hold on the feeling", 200)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


hold on the feeling that the power is gone and the power is ready to stand on my own two knees yes i should know better blue i know youre yours in the night of the parking you used to make me feel it makes my soul my soul my love love love love love love love will not let you down my love love love love love love love love will not let you down my love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love


In [ ]:
make_lyrics("somebody else",100)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


somebody else would help you through the top you think youve got the whole soul you take away the breath of my head i miss you like crazy i miss you baby no matter what i say or ever oh take my only love to disconnect to you you make my dreams come true make you make my love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love love


In [ ]:
make_lyrics("act like summer", 50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


act like summer ta na na na muchawa ta na na na muchawa muchawa ta na na na muchawa muchawa ta na na na muchawa muchawa ta na na na muchawa muchawa ta na na na muchawa muchawa ta na na na muchawa muchawa ta na na na muchawa muchawa ta na na


In [ ]:
make_lyrics("walk like rain", 50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


walk like rain i am told me not to worry about my life my stupid heart is pounding the only beat you a beat you a beat of my heart you know soul of my soul heart of my heart the greatest treasure of my life thats what you are soul of my


# Change the Padding length to 500

### Pad Sequences

In [ ]:
from keras.preprocessing.sequence import pad_sequences

max_sequence_len = 500
inp_sequences = np.array(pad_sequences(inp_sequences,
                       maxlen = max_sequence_len, padding='pre'))

In [ ]:
print(inp_sequences.shape)
inp_sequences

(101732, 500)


array([[   0,    0,    0, ...,    0,   61,  630],
       [   0,    0,    0, ...,   61,  630,   40],
       [   0,    0,    0, ...,  630,   40,    6],
       ...,
       [   0,    0,    0, ..., 1614, 2564,  884],
       [   0,    0,    0, ..., 2564,  884, 1492],
       [   0,    0,    0, ...,  884, 1492, 3526]], dtype=int32)

### Predictions and Labels

we will use our input sequence and use the last word of all sequences as labels for all previous words

In [ ]:
predictors, label = inp_sequences[:,:-1],inp_sequences[:,-1]

In [ ]:
import keras.utils as ku

label = ku.to_categorical(label, num_classes=total_words)

In [ ]:
label.shape

(101732, 6378)

## Build the Model

In [ ]:
from keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model02 = Sequential()
model02.add(Embedding(total_words, 50, input_length=max_sequence_len-1))
# Add a LSTM Layer
model02.add(LSTM(100)) 
model02.add(Dense(total_words/2, activation='relu'))  
# In the last layer, the shape should be equal to the total number of words present in our corpus
model02.add(Dense(total_words, activation='softmax'))
model02.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')  

model02.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 499, 50)           318900    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_2 (Dense)              (None, 3189)              322089    
_________________________________________________________________
dense_3 (Dense)              (None, 6378)              20345820  
Total params: 21,047,209
Trainable params: 21,047,209
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='loss', mode='min', verbose=1)
his02 = model02.fit(predictors, label, epochs= 10, callbacks=[es])

Epoch 1/10
3180/3180 [==============================] - 98s 30ms/step - loss: 6.3857 - accuracy: 0.0461
Epoch 2/10
3180/3180 [==============================] - 96s 30ms/step - loss: 5.3459 - accuracy: 0.1188
Epoch 3/10
3180/3180 [==============================] - 96s 30ms/step - loss: 4.6535 - accuracy: 0.1735
Epoch 4/10
3180/3180 [==============================] - 97s 30ms/step - loss: 4.0517 - accuracy: 0.2299
Epoch 5/10
3180/3180 [==============================] - 96s 30ms/step - loss: 3.4596 - accuracy: 0.3027
Epoch 6/10
3180/3180 [==============================] - 96s 30ms/step - loss: 2.9119 - accuracy: 0.3783
Epoch 7/10
3180/3180 [==============================] - 97s 30ms/step - loss: 2.4045 - accuracy: 0.4578
Epoch 8/10
3180/3180 [==============================] - 97s 30ms/step - loss: 1.9150 - accuracy: 0.5498
Epoch 9/10
3180/3180 [==============================] - 96s 30ms/step - loss: 1.5260 - accuracy: 0.6291
Epoch 10/10
3180/3180 [==============================] - 97s 30m

Save the model

In [ ]:
!pip install -q pyyaml h5py

model02.save('rock_padding500.h5')

### Lyrics Generation

In [ ]:
def rock2_lyrics(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],
                     maxlen=max_sequence_len-1,padding='pre')
        
        predicted = model02.predict_classes(token_list, verbose=0)

        output_word = ""
        
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text)

In [ ]:
rock2_lyrics("look at", 100)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


look at mei must be absolutely crazy how crazy i love you say goodbye if you aint livin if you dont know about lonely till its chiseled in stone you know about lonely till its chiseled in stone you know about lonely till the lights just call yourself this ride running through snow where the vaults are full and the fire comes are the long book i see the less the less i believe the less i believe the less i believe the more i see the less i believe the more i see the less i believe the more i see


In [ ]:
rock2_lyrics("look at", 50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


look at mei must be absolutely crazy how crazy i love you say goodbye if you aint livin if you dont know about lonely till its chiseled in stone you know about lonely till its chiseled in stone you know about lonely till the lights just call yourself this ride running through


In [ ]:
rock2_lyrics("hold on the feeling", 50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


hold on the feeling theres old enough to arrive but if he cant have you guess that no one else will watch him think a while smoke a cigarette and smile as i frown drink about the heartache that cost and he has been himhe like her but she wasnt like that then i


In [ ]:
rock2_lyrics("somebody else",100)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


somebody else was the longest tuff goodbye so many loves and no use besides the sun wont let the show em em oh oh yeah oh yeah oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh youre never far away oh oh oh oh youre never far away oh oh oh oh youre never far away oh oh oh oh youre never far away oh oh oh oh youre never far away oh oh oh oh youre never far away oh


In [ ]:
rock2_lyrics("act like summer", 50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


act like summer my own friends tell me im tossed and moving on a twenty can taste and i cant change back for you i cant read it why cant i cant get by im im not like im not gonna get ready to never get ready to get get out get out


In [ ]:
rock2_lyrics("walk like rain", 50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


walk like rain the goods salesman gonna see the sun wont go down but the end moon as he is different than a uk bside in 1980 steal away child into your dreams where nothing else matters were together again together again go so go so tell me im losin you im losin


## More Layers with Early Stopping

In [ ]:
from keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model05 = Sequential()
model05.add(Embedding(total_words, 50, input_length=max_sequence_len-1))
# Add a LSTM Layer
model05.add(LSTM(128, return_sequences=True)) 
# A dropout layer for regularisation
model05.add(Dropout(0.2))
# Add another LSTM Layer
model05.add(LSTM(100)) 
model05.add(Dense(total_words/2, activation='relu'))  
# In the last layer, the shape should be equal to the total number of words present in our corpus
model05.add(Dense(total_words, activation='softmax'))
model05.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')  

model05.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 499, 50)           318900    
_________________________________________________________________
lstm_5 (LSTM)                (None, 499, 128)          91648     
_________________________________________________________________
dropout_1 (Dropout)          (None, 499, 128)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               91600     
_________________________________________________________________
dense_8 (Dense)              (None, 3189)              322089    
_________________________________________________________________
dense_9 (Dense)              (None, 6378)              20345820  
Total params: 21,170,057
Trainable params: 21,170,057
Non-trainable params: 0
__________________________________________

In [ ]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='loss', mode='min', verbose=1)
his05 = model05.fit(predictors, label, epochs= 10, callbacks=[es])


Epoch 1/10
3180/3180 [==============================] - 170s 52ms/step - loss: 6.4897 - accuracy: 0.0397
Epoch 2/10
3180/3180 [==============================] - 166s 52ms/step - loss: 5.9219 - accuracy: 0.0546
Epoch 3/10
3180/3180 [==============================] - 168s 53ms/step - loss: 5.6115 - accuracy: 0.0808
Epoch 4/10
3180/3180 [==============================] - 168s 53ms/step - loss: 5.3429 - accuracy: 0.0968
Epoch 5/10
3180/3180 [==============================] - 168s 53ms/step - loss: 5.1183 - accuracy: 0.1120
Epoch 6/10
3180/3180 [==============================] - 168s 53ms/step - loss: 4.8863 - accuracy: 0.1271
Epoch 7/10
3180/3180 [==============================] - 169s 53ms/step - loss: 4.6350 - accuracy: 0.1478
Epoch 8/10
3180/3180 [==============================] - 170s 53ms/step - loss: 4.3862 - accuracy: 0.1676
Epoch 9/10
3180/3180 [==============================] - 169s 53ms/step - loss: 4.1448 - accuracy: 0.1906
Epoch 10/10
3180/3180 [==============================] 

Save the model

In [ ]:
!pip install -q pyyaml h5py

model05.save('rock_2layer.h5')

### Lyrics Generation

In [ ]:
def multilayer_lyrics(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],
                     maxlen=max_sequence_len-1,padding='pre')
        
        predicted = model05.predict_classes(token_list, verbose=0)

        output_word = ""
        
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text)

In [ ]:
multilayer_lyrics("look at", 50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


look at the moonlight and i am the same and i am a slave i dont know what to do i am i am i am i am i am i am i am i am i am i am i am i am i am i am i am i am


In [ ]:
multilayer_lyrics("hold on the feeling", 50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


hold on the feeling and the stars is turning round and on and on and on and on and on and on and on and on and on and on and on and on and on and on and on and on and on and on and on and on and on and on


In [ ]:
multilayer_lyrics("act like summer",50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


act like summer of the sky and i know i am the luckiest i am the luckiest the best with a fuck you aint no love in the heart of my life and i am the war i can be a vicious young man oh i am a vicious young man oh i


In [ ]:
multilayer_lyrics("walk like rain",50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


walk like rain i am a slave i dont know what to do i am i am i am i am i am i am i am i am i am i am i am i am i am i am i am i am i am i am i am i am


# Statistical Model

In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

texts = []
for s in corpus:
    texts.append(word_tokenize(s))
print(texts[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['its', 'a', 'simple', 'song', 'for', 'simple', 'feeling', 'you', 'see', 'the', 'moon', 'and', 'watch', 'it', 'rise', 'across', 'the', 'continent', 'the', 'night', 'bird', 'sings', 'and', 'somewhere', 'someone', 'hears', 'its', 'cry', 'so', 'disillusioned', 'keep', 'your', 'head', 'down', 'if', 'you', 'do', 'theyll', 'never', 'know', 'youll', 'have', 'no', 'answers', 'to', 'their', 'questions', 'and', 'they', 'will', 'have', 'to', 'let', 'you', 'go', 'and', 'disenfranchised', 'revolution', 'theyll', 'take', 'away', 'by', 'right', 'whats', 'yours', 'and', 'make', 'you', 'martyrs', 'of', 'your', 'own', 'cause', 'when', 'they', 'dont', 'know', 'what', 'cause', 'its', 'for', 'and', 'all', 'deserted', 'stand', 'alerted', 'theyll', 'love', 'you', 'when', 'youre', 'all', 'alone', 'but', 'you', 'find', 'a', 'red', 'rose', 'in', 'the', 'morning', 'light', 'you', 'wait', 'the', 'night', 'and

In [ ]:
from nltk.util import ngrams

print(list(ngrams(texts[0], n=1)))
print(list(ngrams(texts[0], n=2)))
print(list(ngrams(texts[0], n=3)))

[('its',), ('a',), ('simple',), ('song',), ('for',), ('simple',), ('feeling',), ('you',), ('see',), ('the',), ('moon',), ('and',), ('watch',), ('it',), ('rise',), ('across',), ('the',), ('continent',), ('the',), ('night',), ('bird',), ('sings',), ('and',), ('somewhere',), ('someone',), ('hears',), ('its',), ('cry',), ('so',), ('disillusioned',), ('keep',), ('your',), ('head',), ('down',), ('if',), ('you',), ('do',), ('theyll',), ('never',), ('know',), ('youll',), ('have',), ('no',), ('answers',), ('to',), ('their',), ('questions',), ('and',), ('they',), ('will',), ('have',), ('to',), ('let',), ('you',), ('go',), ('and',), ('disenfranchised',), ('revolution',), ('theyll',), ('take',), ('away',), ('by',), ('right',), ('whats',), ('yours',), ('and',), ('make',), ('you',), ('martyrs',), ('of',), ('your',), ('own',), ('cause',), ('when',), ('they',), ('dont',), ('know',), ('what',), ('cause',), ('its',), ('for',), ('and',), ('all',), ('deserted',), ('stand',), ('alerted',), ('theyll',), ('l

In [ ]:
from nltk.lm.preprocessing import padded_everygram_pipeline

train, vocab = padded_everygram_pipeline(3, texts)

In [ ]:
from nltk.lm import MLE

ng_model = MLE(3) 
ng_model.fit(train, vocab)


In [ ]:
print(ng_model.counts['the'])
print(ng_model.counts['are'])
print(ng_model.counts['love'])
print(ng_model.counts['oh'])

print(ng_model.score('the'))
print(ng_model.score('are'))
print(ng_model.score('love'))
print(ng_model.score('oh'))

print(ng_model.counts[['they']]['are'])
print(ng_model.counts[['are']]['they'])

3613
302
534
369
0.03706857635326466
0.003098452825542742
0.005478721221323922
0.003785857922600238
13
2


In [ ]:
text_list = ng_model.generate(15, random_seed=2)
print(' '.join(word for word in text_list))

you who broke my heart some people do not mess with mister inbetween no dont


In [ ]:
text_list = ng_model.generate(50, random_seed=2)
print(' '.join(word for word in text_list))

you who broke my heart some people do not mess with mister inbetween no dont you worry baby im buying if youre my daughter youre my love ill get it back better late than never i saw you saying that you take my woman where she wants to do it


In [ ]:
text_list = ng_model.generate(50, random_seed=1)
print(' '.join(word for word in text_list))

boy who was hanging his head low more trophies and ideas to follow footprints in the darkest place that cow wrote that im doing it all a big joke whatever it is like porcelain yeah </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
